In [123]:
from __future__ import print_function, division
import sys,os
# line 4 and line 5 below are for development purposes and can be removed
qspin_path = os.path.join(os.getcwd(),"../../")
sys.path.insert(0,qspin_path)
from scipy.linalg import expm
from quspin.operators import hamiltonian, commutator, exp_op # Hamiltonians and operators
from quspin.basis import tensor_basis, spin_basis_1d # bases
from quspin.tools.measurements import obs_vs_time # calculating dynamics
from quspin.tools.Floquet import Floquet_t_vec # period-spaced time vector
import numpy as np # general math functions
import matplotlib.pyplot as plt # plotting library

In [124]:
L = 3
M = 0.1
l=1
no_checks = dict(check_pcon=False,check_symm=False,check_herm=False)


In [125]:
basis=spin_basis_1d(L=L,S = str(l))
print(basis)

reference states: 
array index   /   Fock state   /   integer repr. 
      0.         |2 2 2>           26  
      1.         |2 2 1>           25  
      2.         |2 2 0>           24  
      3.         |2 1 2>           23  
      4.         |2 1 1>           22  
      5.         |2 1 0>           21  
      6.         |2 0 2>           20  
      7.         |2 0 1>           19  
      8.         |2 0 0>           18  
      9.         |1 2 2>           17  
     10.         |1 2 1>           16  
     11.         |1 2 0>           15  
     12.         |1 1 2>           14  
     13.         |1 1 1>           13  
     14.         |1 1 0>           12  
     15.         |1 0 2>           11  
     16.         |1 0 1>           10  
     17.         |1 0 0>            9  
     18.         |0 2 2>            8  
     19.         |0 2 1>            7  
     20.         |0 2 0>            6  
     21.         |0 1 2>            5  
     22.         |0 1 1>            4  
     23.   

In [126]:
basis_vectors = []
for index in range(basis.Ns):
    state = np.zeros(basis.Ns)
    state[basis.Ns-index-1] = 1.
    state = state.tolist()
    basis_vectors.append(state)

In [127]:
sigma_01_0 = np.zeros((basis.Ns,basis.Ns))
sigma_12_0 = np.zeros((basis.Ns,basis.Ns))
sigma_02_0 = np.zeros((basis.Ns,basis.Ns))
sigma_01_1 = np.zeros((basis.Ns,basis.Ns))
sigma_12_1 = np.zeros((basis.Ns,basis.Ns))
sigma_02_1 = np.zeros((basis.Ns,basis.Ns))
sigma_01_2 = np.zeros((basis.Ns,basis.Ns))
sigma_12_2 = np.zeros((basis.Ns,basis.Ns))
sigma_02_2 = np.zeros((basis.Ns,basis.Ns))

for i in range(basis.Ns):
    for j in range(basis.Ns):
        
        string_i = basis.int_to_state(basis.Ns-i-1,bracket_notation=False)
        string_j = basis.int_to_state(basis.Ns-j-1,bracket_notation=False)
        #0th link
        if (int(string_i[0]) == 0) & (int(string_j[0]) == 1) & (int(string_i[1]) == int(string_j[1])) & (int(string_i[2]) == int(string_j[2])):
            sigma_01_0[i][j] = 1.
        if (int(string_i[0]) == 1) & (int(string_j[0]) == 0) & (int(string_i[1]) == int(string_j[1])) & (int(string_i[2]) == int(string_j[2])):
            sigma_01_0[i][j] = 1.
            
        if (int(string_i[0]) == 0) & (int(string_j[0]) == 2) & (int(string_i[1]) == int(string_j[1])) & (int(string_i[2]) == int(string_j[2])):
            sigma_02_0[i][j] = 1.
        if (int(string_i[0]) == 2) & (int(string_j[0]) == 0) & (int(string_i[1]) == int(string_j[1])) & (int(string_i[2]) == int(string_j[2])):
            sigma_02_0[i][j] = 1.
            
        if (int(string_i[0]) == 1) & (int(string_j[0]) == 2) & (int(string_i[1]) == int(string_j[1])) & (int(string_i[2]) == int(string_j[2])):
            sigma_12_0[i][j] = 1.
        if (int(string_i[0]) == 2) & (int(string_j[0]) == 1) & (int(string_i[1]) == int(string_j[1])) & (int(string_i[2]) == int(string_j[2])):
            sigma_12_0[i][j] = 1.
        #1st link
        if (int(string_i[1]) == 0) & (int(string_j[1]) == 1) & (int(string_i[0]) == int(string_j[0])) & (int(string_i[2]) == int(string_j[2])):
            sigma_01_1[i][j] = 1.
        if (int(string_i[1]) == 1) & (int(string_j[1]) == 0) & (int(string_i[0]) == int(string_j[0])) & (int(string_i[2]) == int(string_j[2])):
            sigma_01_1[i][j] = 1.
            
        if (int(string_i[1]) == 0) & (int(string_j[1]) == 2) & (int(string_i[0]) == int(string_j[0])) & (int(string_i[2]) == int(string_j[2])):
            sigma_02_1[i][j] = 1.
        if (int(string_i[1]) == 2) & (int(string_j[1]) == 0) & (int(string_i[0]) == int(string_j[0])) & (int(string_i[2]) == int(string_j[2])):
            sigma_02_1[i][j] = 1.
            
        if (int(string_i[1]) == 1) & (int(string_j[1]) == 2) & (int(string_i[0]) == int(string_j[0])) & (int(string_i[2]) == int(string_j[2])):
            sigma_12_1[i][j] = 1.
        if (int(string_i[1]) == 2) & (int(string_j[1]) == 1) & (int(string_i[0]) == int(string_j[0])) & (int(string_i[2]) == int(string_j[2])):
            sigma_12_1[i][j] = 1.
        #2nd link
        if (int(string_i[2]) == 0) & (int(string_j[2]) == 1) & (int(string_i[1]) == int(string_j[1])) & (int(string_i[0]) == int(string_j[0])):
            sigma_01_2[i][j] = 1.
        if (int(string_i[2]) == 1) & (int(string_j[2]) == 0) & (int(string_i[1]) == int(string_j[1])) & (int(string_i[0]) == int(string_j[0])):
            sigma_01_2[i][j] = 1.
            
        if (int(string_i[2]) == 0) & (int(string_j[2]) == 2) & (int(string_i[1]) == int(string_j[1])) & (int(string_i[0]) == int(string_j[0])):
            sigma_02_2[i][j] = 1.
        if (int(string_i[2]) == 2) & (int(string_j[2]) == 0) & (int(string_i[1]) == int(string_j[1])) & (int(string_i[0]) == int(string_j[0])):
            sigma_02_2[i][j] = 1.
            
        if (int(string_i[2]) == 1) & (int(string_j[2]) == 2) & (int(string_i[1]) == int(string_j[1])) & (int(string_i[0]) == int(string_j[0])):
            sigma_12_2[i][j] = 1.
        if (int(string_i[2]) == 2) & (int(string_j[2]) == 1) & (int(string_i[1]) == int(string_j[1])) & (int(string_i[0]) == int(string_j[0])):
            sigma_12_2[i][j] = 1.         
            
sigma_01 = sigma_01_0 + sigma_01_1 + sigma_01_2
sigma_12 = sigma_12_0 + sigma_12_1 + sigma_12_2
sigma_02 = sigma_02_0 + sigma_02_1 + sigma_02_2
def rot(sigma,theta):
    return expm(-1j*theta*sigma/2)

In [128]:
ms_01_0_1 = ((sigma_01_0+sigma_01_1)**2)/2  
ms_12_0_1 = ((sigma_12_0+sigma_12_1)**2)/2
ms_02_0_1 = ((sigma_02_0+sigma_02_1)**2)/2

ms_01_1_2 = ((sigma_01_1+sigma_01_2)**2)/2
ms_12_1_2 = ((sigma_12_1+sigma_12_2)**2)/2
ms_02_1_2 = ((sigma_02_1+sigma_02_2)**2)/2

ms_01 = ms_01_0_1 + ms_01_1_2
ms_12 = ms_12_0_1 + ms_12_1_2
ms_02 = ms_02_0_1 + ms_02_1_2

In [129]:
#Gauss_law_0

gauss_law_z_0 = []
gauss_law_z_0.append([1.,0])

gauss_law_0_map = [
    
    ["z",gauss_law_z_0]

    
]

Gauss_law_0  = hamiltonian(gauss_law_0_map,dynamic_list=[],basis=basis,**no_checks)

constraint_0 = Gauss_law_0**2-Gauss_law_0

In [130]:
Proj_10 = 0
Proj_00 = 0
for state in basis_vectors:
    state = np.array(state)
    if (Gauss_law_0.expt_value(state)>=1.-1e-8) & (Gauss_law_0.expt_value(state)<=1.+1e-8):
        Proj_10 += np.outer(state,state)
    if (Gauss_law_0.expt_value(state)>=0.-1e-8) & (Gauss_law_0.expt_value(state)<=0.+1e-8):
        Proj_00 += np.outer(state,state)

In [131]:
#Gauss_law_1
gauss_law_zm_1 = []
gauss_law_zm_1.append([-1.,0])

gauss_law_z_1 = []
gauss_law_z_1.append([1.,1])

gauss_law_id_1 = [[1.,1]]

gauss_law_1_map = [
    
    ["z",gauss_law_z_1],
    ["z",gauss_law_zm_1],
    ["I",gauss_law_id_1]
    
]

Gauss_law_1  = hamiltonian(gauss_law_1_map,dynamic_list=[],basis=basis,**no_checks)
constraint_1 = Gauss_law_1**2-Gauss_law_1

In [132]:
Proj_01 = 0
Proj_11 = 0
for state in basis_vectors:
    state = np.array(state)
    if (Gauss_law_1.expt_value(state)>=0.-1e-8) & (Gauss_law_1.expt_value(state)<=0.+1e-8):
        Proj_01 += np.outer(state,state)
    if (Gauss_law_1.expt_value(state)>=1.-1e-8) & (Gauss_law_1.expt_value(state)<=1.+1e-8):
        Proj_11 += np.outer(state,state)

In [133]:
#Gauss_law_2
gauss_law_zm_2 = []
gauss_law_zm_2.append([-1.,1])

gauss_law_z_2 = []
gauss_law_z_2.append([1.,2])


gauss_law_2_map = [
    
    ["z",gauss_law_z_2],
    ["z",gauss_law_zm_2],
    
]

Gauss_law_2  = hamiltonian(gauss_law_2_map,dynamic_list=[],basis=basis,**no_checks)
constraint_2 = Gauss_law_2**2-Gauss_law_2

In [134]:
Proj_02 = 0
Proj_12 = 0
for state in basis_vectors:
    state = np.array(state)
    if (Gauss_law_2.expt_value(state)>=0.-1e-8) & (Gauss_law_2.expt_value(state)<=0.+1e-8):
        Proj_02 += np.outer(state,state)
    if (Gauss_law_2.expt_value(state)>=1.-1e-8) & (Gauss_law_2.expt_value(state)<=1.+1e-8):
        Proj_12 += np.outer(state,state)

In [135]:
#Gauss_law_3

gauss_law_z_3 = []
gauss_law_z_3.append([-1.,2])

gauss_law_id_3 = [[1.,2]]

gauss_law_3_map = [
    
    ["z",gauss_law_z_3],
    ["I",gauss_law_id_3]

    
]

Gauss_law_3  = hamiltonian(gauss_law_3_map,dynamic_list=[],basis=basis,**no_checks)
constraint_3 = Gauss_law_3**2-Gauss_law_3

In [136]:
Proj_03 = 0
Proj_13 = 0

for state in basis_vectors:
    state = np.array(state)
    if (Gauss_law_3.expt_value(state)>=0.-1e-8) & (Gauss_law_3.expt_value(state)<=0.+1e-8):
        Proj_03 += np.outer(state,state)
    if (Gauss_law_3.expt_value(state)>=1.-1e-8) & (Gauss_law_3.expt_value(state)<=1.+1e-8):
        Proj_13 += np.outer(state,state)

In [137]:
linear_term_0 = [[0.5/np.sqrt(2),0]]
quadratic_term_0 = [[1./np.sqrt(2),0,1]]

linear_term_1 = [[0.5/np.sqrt(2),1]]
quadratic_term_1 = [[-1./np.sqrt(2),1,2]]

linear_term_2 = [[0.5/np.sqrt(2),2]]

kin_energy = [[0.5,i,i] for i in range(L)]
mass_term = []
mass_term.append([2*M,0])
mass_term.append([-2*M,1])
mass_term.append([2*M,2])

kin_mass_map = [
            ["zz", kin_energy], 
            ["z", mass_term], 
]

interaction_map_p1 = [
    
    ["+z", quadratic_term_1],
    ["+", linear_term_1]
]

interaction_map_m1 = [
    
    ["-z", quadratic_term_1],
    ["-", linear_term_1]
]

interaction_map_p0 = [
    
    ["+", linear_term_0],
    ["+z", quadratic_term_0],

]

interaction_map_m0 = [
    
    ["-", linear_term_0],
    ["-z", quadratic_term_0],
]

interaction_map_p2 = [
    
    ["+", linear_term_2]
]

interaction_map_m2 = [
    
    ["-", linear_term_2]
]


H_int_p0 = hamiltonian(interaction_map_p0,dynamic_list=[],basis=basis,**no_checks)
H_int_m0 = hamiltonian(interaction_map_m0,dynamic_list=[],basis=basis,**no_checks)

H_int_p1 = hamiltonian(interaction_map_p1,dynamic_list=[],basis=basis,**no_checks)
H_int_m1 = hamiltonian(interaction_map_m1,dynamic_list=[],basis=basis,**no_checks)

H_int_p2 = hamiltonian(interaction_map_p2,dynamic_list=[],basis=basis,**no_checks)
H_int_m2 = hamiltonian(interaction_map_m2,dynamic_list=[],basis=basis,**no_checks)


H_kin_mass = hamiltonian(kin_mass_map,dynamic_list=[],basis=basis,**no_checks)

h_int_p0 = H_int_p0.toarray()
h_int_m0 = H_int_m0.toarray()

h_int_p1 = H_int_p1.toarray()
h_int_m1 = H_int_m1.toarray()


h_int_p2 = H_int_p2.toarray()
h_int_m2 = H_int_m2.toarray()


h_kin_mass = H_kin_mass.toarray()

h_int_0 = Proj_10 @ h_int_p0 @ Proj_11 + Proj_11 @ h_int_m0 @ Proj_10
h_int_1 = Proj_11 @ h_int_p1 @ Proj_12 + Proj_12 @ h_int_m1 @ Proj_11
h_int_2 = Proj_12 @ h_int_p2 @ Proj_13 + Proj_13 @ h_int_m2 @ Proj_12

h_int = h_int_0+h_int_1+h_int_2

full_ham_matrix = h_kin_mass + h_int
constraint_full = 1e3*(constraint_0**2+constraint_1**2+constraint_2**2+constraint_3**2)
constraint_full_matrix = constraint_full.toarray()

restricted_full_ham_matrix = full_ham_matrix+constraint_full_matrix


In [138]:
eigenval,eigenvec = np.linalg.eigh(full_ham_matrix)

In [139]:
print(eigenval)

[-0.64551385  0.03967222  0.03967222  0.3         0.42748942  0.6
  0.6         0.6         0.7         0.9         0.976925    1.
  1.          1.          1.          1.26032778  1.26032778  1.3
  1.3         1.3         1.4         1.4         1.49754183  1.7
  1.7         2.1         2.24355759]


In [140]:
eigenval_restricted,eigenvec_restricted = np.linalg.eigh(restricted_full_ham_matrix)

In [141]:
print(eigenval_restricted[eigenval_restricted<=1e2])

[-0.64551385  0.42748942  0.7         0.976925    1.49754183  2.24355759]


In [142]:
psi_0 = np.zeros(basis.Ns)
i_0 = basis.index("000")
psi_0[i_0] = 1.

In [143]:
expt_value = 0
matvec_h = full_ham_matrix@psi_0
matvec_c = constraint_full_matrix@psi_0
for i in range(basis.Ns):
    expt_value += np.conj(psi_0[i])*(matvec_h[i]+matvec_c[i])
print(np.real(expt_value))

8001.3


In [144]:
expt_value = 0
matvec = full_ham_matrix@psi_0
for i in range(basis.Ns):
    expt_value += np.conj(psi_0[i])*matvec[i]
print(expt_value)

(1.3+0j)


In [145]:
def cost_function_sigma(theta,psi,a):

    psi_1 = rot(sigma_02,theta[2])@rot(sigma_12,theta[1])@rot(sigma_01,theta[0])@psi
    
    psi_2 = rot(ms_02,theta[5])@rot(ms_12,theta[4])@rot(ms_01,theta[3])@psi_1
    
    psi_3 = rot(sigma_02,theta[8])@rot(sigma_12,theta[7])@rot(sigma_01,theta[6])@psi_2
    
    psi_4 = rot(ms_02,theta[11])@rot(ms_12,theta[10])@rot(ms_01,theta[9])@psi_3
    
    psi_5 = rot(sigma_02,theta[14])@rot(sigma_12,theta[13])@rot(sigma_01,theta[12])@psi_4
    
    psi_6 = rot(ms_02,theta[17])@rot(ms_12,theta[16])@rot(ms_01,theta[15])@psi_5
    
    psi_7 = rot(sigma_02,theta[20])@rot(sigma_12,theta[19])@rot(sigma_01,theta[18])@psi_6
    
    psi_8 = rot(ms_02,theta[23])@rot(ms_12,theta[22])@rot(ms_01,theta[21])@psi_7
    
    psi_9 = rot(sigma_02,theta[26])@rot(sigma_12,theta[25])@rot(sigma_01,theta[24])@psi_8
    


    expt_value = 0
    matvec = full_ham_matrix@psi_9
    for i in range(basis.Ns):
        expt_value += np.conj(psi_9[i])*matvec[i]
    return np.real(expt_value)

In [146]:
from scipy.optimize import dual_annealing
import datetime
duan_ranges = [(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.)]
resduan = 0
resopt = 0
print(datetime.datetime.now())
resduan = dual_annealing(cost_function_sigma, duan_ranges, args = (psi_0,1))
print(datetime.datetime.now())

2022-02-16 11:46:06.728181
2022-02-16 12:01:06.421920


In [147]:
print(resduan.fun)
print(resduan.x)

-0.5914939400832884
[-1.48456361  6.95442669  2.2280383  -2.69906652  3.38951284  9.54468822
 -5.20389126  6.72627137  0.38821034  4.56212042  6.39630267  4.03749505
 -4.57560968  6.05682194 -7.7623525  -3.31534537 -5.04840219  3.40898486
 -6.9363534  -8.56678541 -8.95021911 -0.62446287  8.76660172  2.41620271
 -6.91681003  6.6804874   1.67071512]


In [148]:
theta = resduan.x
psi = psi_0

psi_1 = rot(sigma_02,theta[2])@rot(sigma_12,theta[1])@rot(sigma_01,theta[0])@psi

psi_2 = rot(ms_02,theta[5])@rot(ms_12,theta[4])@rot(ms_01,theta[3])@psi_1

psi_3 = rot(sigma_02,theta[8])@rot(sigma_12,theta[7])@rot(sigma_01,theta[6])@psi_2

psi_4 = rot(ms_02,theta[11])@rot(ms_12,theta[10])@rot(ms_01,theta[9])@psi_3

psi_5 = rot(sigma_02,theta[14])@rot(sigma_12,theta[13])@rot(sigma_01,theta[12])@psi_4

psi_6 = rot(ms_02,theta[17])@rot(ms_12,theta[16])@rot(ms_01,theta[15])@psi_5

psi_7 = rot(sigma_02,theta[20])@rot(sigma_12,theta[19])@rot(sigma_01,theta[18])@psi_6

psi_8 = rot(ms_02,theta[23])@rot(ms_12,theta[22])@rot(ms_01,theta[21])@psi_7

psi_9 = rot(sigma_02,theta[26])@rot(sigma_12,theta[25])@rot(sigma_01,theta[24])@psi_8

print(np.abs(np.dot(np.conj(psi_9),eigenvec[:,0]))**2)
print(psi_9)
print(np.abs(np.dot(np.conj(psi_9),psi_9)))

0.9712225614843492
[ 1.66805537e-03-5.39221326e-04j -3.83768958e-03+1.24060268e-03j
 -1.73980484e-08+3.52236867e-09j  1.47213220e-01-4.76230108e-02j
 -3.38692979e-01+1.09567689e-01j -1.53549348e-06+3.11210210e-07j
 -3.43949531e-02+1.11270366e-02j  7.91323572e-02-2.56003066e-02j
  3.58753716e-07-7.27150763e-08j -3.83768958e-03+1.24060268e-03j
  8.82935999e-03-2.85429181e-03j  4.00276550e-08-8.10408247e-09j
 -3.38692979e-01+1.09567689e-01j  7.79229839e-01-2.52085666e-01j
  3.53270668e-06-7.16016238e-07j  7.91323572e-02-2.56003066e-02j
 -1.82059558e-01+5.88993927e-02j -8.25383933e-07+1.67299060e-07j
 -1.73980482e-08+3.52236892e-09j  4.00276551e-08-8.10408247e-09j
  1.79113668e-13-1.53141388e-14j -1.53549348e-06+3.11210210e-07j
  3.53270668e-06-7.16016238e-07j  1.58036466e-11-1.37950762e-12j
  3.58753716e-07-7.27150761e-08j -8.25383933e-07+1.67299060e-07j
 -3.69255222e-12+3.22052464e-13j]
0.9999999999999976


In [149]:
expt_value = 0
matvec_h = full_ham_matrix@psi_9
matvec_c = constraint_full_matrix@psi_9
for i in range(basis.Ns):
    expt_value += np.conj(psi_9[i])*(matvec_h[i]+matvec_c[i])
print(np.real(expt_value))

66.0205137651833
